# 5DEEP - Projet final : Classification des chants d'oiseaux

Pour rappel:

Il s'agit d'un problème de classification de chants d'oiseaux avec 5 classes, une pour chacune des espèces suivantes : 

- Bewick's Wren
- Northern Cardinal
- American Robin
- Song Sparrow
- Northern Mockingbird

Pour répondre à cette problématique, nous utiliserons des réseaux de neuronnes que nous préparerons et entraînerons grâce à Keras et Librosa dans ce notebook Jupyter.

## Etape 1: Analyse exploratoire et comprehension des données

Nous allons tout d'abord commencer par comprendre les différentes données mises à disposition, mais aussi comprendre l'interpretation de ces fichiers audio et leur traitement.

In [ ]:
import pandas as pd

bird_df = pd.read_csv("data/bird_songs_metadata.csv")

bird_df.head(10)
